In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle 

In [6]:
## Load the dataset
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


# Feature Engineering

In [7]:
## Preprocess Data
## Drop irrelevant data
data = data.drop(['RowNumber','CustomerId','Surname'], axis = 1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
## Encode categorical variables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


We cant do Label encoding to countries , because if we do so , it will give numeric values to each coutry say france 0 , spain 1 , germany 2 , so ANN might consider germany(2) greater than france(0)

In [9]:
## Onehot encoding Geography
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [10]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [11]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns = onehot_encoder_geo.get_feature_names_out(['Geography']))

We had to convert geo_encoder into an array because the output of OneHotEncoder is a sparse matrix by default, which is efficient for storage but not easy to view or use directly.

By calling .toarray(), we convert the sparse matrix into a regular NumPy array, making it compatible with pandas and easier to inspect, visualize, and use for further processing or modeling. This step is especially useful when we want to create a DataFrame or need the encoded data in a standard array format.

In [12]:
## Combine one hot encoded columns with the original data
data = pd.concat([data.drop('Geography',axis = 1),geo_encoded_df],axis = 1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [13]:
## Save the encoders
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [14]:
## Divide the dataset into dependent and independent features
## Exited is dependent , other all are independent
## Basically setting Exited as the target feature

X = data.drop('Exited',axis = 1)
y = data['Exited']

## Split dataset into training and testing set
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.8,random_state=42)

## Scale these features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

##  For the test data, we use only transform (not fit) so that the test data is scaled using the same parameters learned from 
##  the training data. This ensures that both training and test data are scaled consistently.

In [15]:
## Save the scaler 
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

# ANN Implementation

In [16]:
%pip install tensorflow

In [17]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [18]:
## Build our ANN model
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu'), ## HL1 connected with input layer
    Dense(32, activation='relu'), ## HL2
    Dense(1 , activation='sigmoid') ## output later
])

In [19]:
## Compile the model (set optimiser , loss fn , metrics)

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

## Can be done like this as well if we want customised optimisers , loss , etc
## opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
## loss = tensorflow.keras.losses.BinaryCrossentropy()
## model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])

In [20]:
## Set up the Tensorboard
log_dir = 'logs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [21]:
## Set up Early Stopping
## Suppose if we set out model to train for 100 epochs , but after say 20 epochs the model has been train to its optimal form , the loss fn is not decreasing anymore , we dont run 
## the next epochs , so basically we early stop

early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [22]:
## Train the model
history = model.fit(
    X_train,y_train,validation_data = (X_test, y_test), 
    epochs = 100 , 
    callbacks = [tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7905 - loss: 0.4992 - val_accuracy: 0.8016 - val_loss: 0.4561
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7905 - loss: 0.4992 - val_accuracy: 0.8016 - val_loss: 0.4561
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8180 - loss: 0.4342 - val_accuracy: 0.8104 - val_loss: 0.4369
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8180 - loss: 0.4342 - val_accuracy: 0.8104 - val_loss: 0.4369
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8280 - loss: 0.4163 - val_accuracy: 0.8125 - val_loss: 0.4275
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8280 - loss: 0.4163 - val_accuracy: 0.8125 - val_loss: 0.4275
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8290 - loss: 0.4034 - val_accuracy: 0.8200 - val_loss: 0.4189
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8290 - loss: 0.4034 - val_accuracy: 0.8200 - v

In [23]:
model.save('model.keras')

In [24]:
## load tensorboard extension
%load_ext tensorboard

In [25]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 18348), started 6:47:27 ago. (Use '!kill 18348' to kill it.)